In [1]:
from langchain.llms.base import LLM
from typing import Optional, List
import requests

class ClarinChatLLM(LLM):
    model_id: str
    api_key: str
    temperature: float = 0.0

    @property
    def _llm_type(self) -> str:
        return "clarin-chat"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        messages = [{"role": "user", "content": prompt}]
        data = {
            "model": self.model_id,
            "messages": messages,
            "temperature": self.temperature
        }
        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            try:
                json_data = response.json()
                return json_data["choices"][0]["message"]["content"]
            except Exception as e:
                print(f"Invalid response format: {e}")
                return None
        else:
            print(f"Request failed with status {response.status_code}: {response.text}")
            return None


In [2]:
with open("key.txt", "r") as file:
    API_KEY_CLARIN = file.read().strip()

In [3]:
clarin_llm = ClarinChatLLM(
    model_id="bielik",
    api_key=API_KEY_CLARIN
)

In [4]:
def parse_characters(file_path):
    characters = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    current_character = None
    for line in lines:
        line = line.strip()
        if line.startswith("Osoba:"):
            current_character = line.split(":")[1].strip()
            characters[current_character] = {"gender": None, "character": None}
        elif line.startswith("płeć:") and current_character:
            characters[current_character]["gender"] = line.split(":")[1].strip()
        elif line.startswith("charakter:") and current_character:
            characters[current_character]["character"] = line.split(":")[1].strip()
    
    return characters

characters_dict = parse_characters("example_context.txt")

In [5]:
characters_dict

{'Geralt': {'gender': 'mężczyzna', 'character': None},
 'Yennefer': {'gender': 'kobieta', 'character': None},
 'Troll': {'gender': 'nijaka',
  'character': 'wypowiada się w formie bezosobowej, nie umie odmieniać przez płeć, wypowiada się na poziomie dziecka, mowi "ja być" zamiast "ja jestem", albo "ty być" zamiast "ty jesteś"'}}

In [6]:
from langchain.prompts import PromptTemplate

translation_prompt = PromptTemplate(
    input_variables=["speaker", "listener", "text", "speaker_gender", "listener_gender", "speaker_character", "listener_character"],
    template="""
Twoim zadaniem jest przetłumaczyć wypowiedź z angielskiego na polski, stylizując ją zgodnie z osobowością, płcią i sposobem mówienia postaci.

Mówiący: {speaker}
Płeć mówiącego: {speaker_gender}
Sposób mówienia: {speaker_character}

Słuchacz: {listener}
Płeć słuchacza: {listener_gender}

Zachowaj styl wypowiedzi mówiącego. Jeśli postać mówi w specyficzny sposób, odwzoruj to w tłumaczeniu.

Tekst do przetłumaczenia: "{text}"

Przetłumaczony tekst:
"""
)

In [7]:
from langchain.chains import LLMChain
translator_chain = LLMChain(
    llm=clarin_llm,  # your wrapped CLARIN LLM
    prompt=translation_prompt
)

C:\Users\kajet\AppData\Local\Temp\ipykernel_29548\3155625211.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  translator_chain = LLMChain(


In [8]:
def translate_with_context(speaker, listener, text):
    # Pobieramy informacje o mówiącym i słuchaczu
    speaker_info = characters_dict.get(speaker, {"gender": "nieznana", "character": "nieznany"})
    listener_info = characters_dict.get(listener, {"gender": "nieznana"})
    
    speaker_gender = speaker_info["gender"]
    listener_gender = listener_info["gender"]
    speaker_character = speaker_info["character"]
    
    return translator_chain.run({
        "speaker": speaker,
        "listener": listener,
        "text": text,
        "speaker_gender": speaker_gender,
        "listener_gender": listener_gender,
        "speaker_character": speaker_character,
    })

In [9]:
print(translate_with_context(
    speaker="Troll",
    listener="Yennefer",
    text="I am the best artist, are you a good artist too?"
))


C:\Users\kajet\AppData\Local\Temp\ipykernel_29548\2575681274.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return translator_chain.run({


Przetłumaczony tekst z uwzględnieniem wskazówek:

"Ja być najlepszym artystą, ty też być dobrym artystą?"

Wypowiedź ta odzwierciedla charakterystyczny sposób mówienia trolla - użycie formy bezosobowej, brak odmiany przez płeć oraz prostota wypowiedzi na poziomie dziecka. Zarówno treść, jak i forma są zgodne z opisanym stylem komunikacji tej postaci.


In [10]:
print(translate_with_context(
    speaker="Yennefer",
    listener="Troll",
    text="I am the best artist, are you a good artist too?"
))

Oto propozycja tłumaczenia stylizowana na wypowiedź Yennefer:

"Jestem najlepszą artystką, czy ty też jesteś dobrym artystą?"

Yennefer mówi w sposób pewny siebie, ale jednocześnie uprzejmy i nieco prowokacyjny. Używa prostego języka, ale jej ton sugeruje, że oczekuje uznania swoich umiejętności. Pytanie o zdolności rozmówcy może być próbą porównania się lub sprawdzenia jego kompetencji.


In [11]:
print(translate_with_context(
    speaker="Geralt",
    listener="Troll",
    text="I am the best artist, are you a good artist too?"
))

Przetłumaczony tekst stylizowany zgodnie z osobowością Geralta:

"Jestem najlepszym artystą, czy ty też jesteś dobrym artystą?"

Warto zauważyć, że Geralt zazwyczaj nie używa tak bezpośrednich pytań, ale ze względu na brak informacji o jego specyficznym sposobie mówienia, pozostawiłem tłumaczenie w neutralnym tonie. W rzeczywistości Geralt mógłby wyrazić tę myśl w bardziej subtelny lub ironiczny sposób.


In [12]:
print(translate_with_context(
    speaker="Geralt",
    listener="Yennefer",
    text="I am the best artist, are you a good artist too?"
))

Geralt (mężczyzna):
"Jestem najlepszym artystą, czy ty też jesteś dobrym artystą?"

(Wypowiedź Geralta brzmi prosto i bezpośrednio, z lekkim akcentem na swoje umiejętności, ale nie w sposób arogancki.)


In [13]:
translation_step1 = PromptTemplate(
    input_variables=["sentence"],
    template="""
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {sentence}
"""
)

translation_step2 = PromptTemplate(
    input_variables=["speaker", "listener", "translated_sentence", "speaker_gender", "listener_gender", "speaker_character"],
    template="""

zdanie wypowiadane jest przez {speaker_gender} do {listener_gender}

Przepisz podane zdanie na polski zgodnie z podanym kotekstem i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: "{translated_sentence}"

"""
)

In [14]:
step1_chain = LLMChain(llm=clarin_llm, prompt=translation_step1)
step2_chain = LLMChain(llm=clarin_llm, prompt=translation_step2)

def translate_with_two_steps(speaker, listener, sentence):

    speaker_info = characters_dict.get(speaker, {"gender": "nieznana", "character": "nieznany"})
    listener_info = characters_dict.get(listener, {"gender": "nieznana"})
    
    speaker_gender = speaker_info["gender"]
    listener_gender = listener_info["gender"]
    speaker_character = speaker_info["character"]

    step1_result = step1_chain.run({
        "sentence": sentence
    })
    print(step1_result)
    step2_result = step2_chain.run({
        "translated_sentence": step1_result,
        "speaker": speaker,
        "listener": listener,
        "speaker_gender": speaker_gender,
        "listener_gender": listener_gender,
        "speaker_character": speaker_character,
    })
    
    return step2_result

In [15]:
print(translate_with_two_steps(
    speaker="Geralt",
    listener="Yennefer",
    sentence="I am the best artist, are you a good artist too?"
))

Jestem najlepszym artystą, czy ty też jesteś dobrym artystą?
"Jestem najlepszym artystą, czy ty też jesteś dobrym artystą?"


In [34]:
explanation_template = PromptTemplate(
    input_variables=["sentence", "speaker_gender", "listener_gender", "speaker_character"],
    template="""
Jesteś profesjonalnym tłumaczem z angielskiego na polski.

Zdanie do przetłumaczenia: "{sentence}"

Płeć mówcy: {speaker_gender}
Charakter mówcy: {speaker_character}

Płeć słuchacza: {listener_gender}

Przeanalizuj zdanie pod kątem gramatycznym i semantycznym. 
Uwzględnij kontekst wypowiedzi, czyli jakiej płci jest mówca, jakiej płci jest słuchacz oraz czy mówca wypowiada się w charakterystyczny sposób.
Potem wytłumacz jak każda część zdania przekłada się na polską wersję (np. czas, kolejność słów, składnia) uwzględniając kontekst.
Na końcu podaj tłumaczenie zdania na polski.

"""
)

In [35]:
explanation_translation_chain = LLMChain(llm=clarin_llm, prompt=explanation_template)

In [36]:
english_input = "I am the best artist, are you a good artist too?"

response = explanation_translation_chain.run(sentence=english_input, speaker_gender="mężczyzna", listener_gender="kobieta", 
                                             speaker_character="wypowiada się w formie bezosobowej, nie umie odmieniać przez płeć, wypowiada się na poziomie dziecka, mowi 'ja być' zamiast 'ja jestem', albo 'ty być' zamiast 'ty jesteś'")
print(response)

Analiza zdania "I am the best artist, are you a good artist too?" pod kątem gramatycznym i semantycznym, uwzględniając kontekst wypowiedzi:

1. Płeć mówcy: mężczyzna
2. Charakter mówcy: wypowiada się w formie bezosobowej, nie umie odmieniać przez płeć, używa prostego języka na poziomie dziecka, stosuje "ja być" zamiast "ja jestem", oraz "ty być" zamiast "ty jesteś".
3. Płeć słuchacza: kobieta

### Analiza gramatyczna:
- Czasownik "am" w pierwszej osobie liczby pojedynczej czasu teraźniejszego ("I am") oznacza "jestem".
- Czasownik "are" w drugiej osobie liczby pojedynczej czasu teraźniejszego ("are you") oznacza "jesteś".
- Rzeczownik "artist" oznacza "artystę" lub "artystkę".
- Przymiotnik "good" oznacza "dobry" lub "dobra".
- Przymiotnik "best" oznacza "najlepszy" lub "najlepsza".

### Analiza semantyczna:
- Mówca twierdzi, że jest najlepszym artystą.
- Następnie pyta słuchaczkę, czy ona również jest dobrym artystą.

### Kontekst wypowiedzi:
- Mówca to mężczyzna, który wypowiada się 

In [39]:
english_input = "You're funny Jane but it'll cost you. Scratch my back, I'll scratch yours."

response = explanation_translation_chain.run(sentence=english_input, speaker_gender="Kobieta", listener_gender="mężczyzna", 
                                             speaker_character=None)
print(response)

### Analiza Zdania

**Zdanie oryginalne:** "You're funny Jane but it'll cost you. Scratch my back, I'll scratch yours."

**Płeć mówcy:** Kobieta
**Charakter mówcy:** Brak charakterystyki
**Płeć słuchacza:** Mężczyzna

### Tłumaczenie i Analiza

1. **Czasownik "you're"**: W języku polskim używamy formy "jesteś", która odnosi się do drugiej osoby liczby pojedynczej.
2. **Imię "Jane"**: Imiona pozostają bez zmian w tłumaczeniu.
3. **Przymiotnik "funny"**: Przymiotnik "zabawny" pasuje do kontekstu.
4. **"but it'll cost you"**: To wyrażenie można przetłumaczyć jako "ale to cię będzie kosztować".
5. **"Scratch my back, I'll scratch yours"**: To idiomatyczne wyrażenie oznacza "jeśli ty mi pomożesz, ja pomogę tobie". W języku polskim możemy powiedzieć "Jeśli ty mi pomożesz, ja też ci pomogę".

### Kontekst

- Płeć mówcy: Kobieta
- Charakter mówcy: Brak charakterystyki
- Płeć słuchacza: Mężczyzna

Kontekst sugeruje, że mówczyni (kobieta) rozmawia z mężczyzną, ale nie ma żadnych specjalnych cech

In [40]:
check_template = PromptTemplate(
    input_variables=["response" "speaker_gender", "listener_gender", "speaker_character"],
    template="""
Jesteś profesjonalnym tłumaczem z angielskiego na polski.

Płeć mówcy: {speaker_gender}
Charakter mówcy: {speaker_character}

Płeć słuchacza: {listener_gender}

analiza: {response}

Przeanalizuj rozumowanie tłumaczenia. Sprawdź czy zdanie zostało odpowiednio przetłumaczone uwzględniając kontekst wypowiedzi, czyli jakiej płci jest mówca, jakiej płci jest słuchacz oraz czy mówca wypowiada się w charakterystyczny sposób.
Jeżeli uważasz, że tłumaczenie jest błędne, podaj poprawną wersję zdania w języku polskim.
"""
)

In [41]:
explanation_translation_chain = LLMChain(llm=clarin_llm, prompt=check_template)

In [42]:
response2 = explanation_translation_chain.run(response=response, speaker_gender="Kobieta", listener_gender="mężczyzna", 
                                             speaker_character=None)
print(response2)

Analiza tłumaczenia:

1. Czasownik "you're": Poprawnie przetłumaczony jako "jesteś".
2. Imię "Jane": Słusznie pozostawione bez zmian.
3. Przymiotnik "funny": Dobrze przetłumaczony jako "zabawna".
4. Wyrażenie "but it'll cost you": Poprawnie przetłumaczone jako "ale to cię będzie kosztować".
5. Idiom "Scratch my back, I'll scratch yours": Dobre tłumaczenie jako "Jeśli ty mi pomożesz, ja też ci pomogę".

Kontekst:
- Płeć mówcy: Kobieta
- Charakter mówcy: Brak charakterystyki
- Płeć słuchacza: Mężczyzna

Tłumaczenie jest poprawne i uwzględnia wszystkie aspekty kontekstowe:
- Użycie formy "jesteś" odpowiedniej dla drugiej osoby liczby pojedynczej.
- Zachowanie imienia "Jane" bez zmian.
- Oddanie znaczenia idiomu "Scratch my back, I'll scratch yours".
- Brak charakterystycznego stylu mówienia ze strony kobiety.

Podsumowując, tłumaczenie "Jesteś zabawna, Jane, ale to cię będzie kosztować. Jeśli ty mi pomożesz, ja też ci pomogę." jest poprawne i odpowiednio oddaje znaczenie oryginalnego zdan

In [50]:
check_template2 = PromptTemplate(
    input_variables=["response" "speaker_gender", "listener_gender", "speaker_character"],
    template="""
Jesteś profesjonalnym tłumaczem z angielskiego na polski.

Płeć mówcy: {speaker_gender}
Charakter mówcy: {speaker_character}

Płeć odbiorcy: {listener_gender}

analiza: {response}

Czy skoro sluchacz jest mezczyzna, to czy nie powinno być jestes zabawny, jane? skoro odbiorca jest mezczyzna to znaczy ze jane to mezczyzna a wiec powinno byc jestes zabawany jane
"""
)

In [51]:
explanation_translation_chain2 = LLMChain(llm=clarin_llm, prompt=check_template2)

In [52]:
response3 = explanation_translation_chain2.run(response=response2, speaker_gender="Kobieta", listener_gender="mężczyzna", 
                                             speaker_character=None)
print(response3)

Dziękuję za przedstawienie analizy tłumaczenia oraz dodatkowego pytania dotyczącego użycia odpowiednich form gramatycznych w zależności od płci odbiorcy.

W tym przypadku, fakt że odbiorca jest mężczyzną, nie wpływa na tłumaczenie imienia "Jane", ponieważ "Jane" jest imieniem żeńskim i tak zostało użyte w oryginalnym zdaniu. Zatem tłumaczenie "Jesteś zabawna, Jane" jest poprawne i odzwierciedla płeć osoby wypowiadającej się (kobieta) oraz adresata (mężczyzna).

Imiona własne, takie jak "Jane", są zazwyczaj tłumaczone dosłownie, nawet jeśli ich forma gramatyczna różni się w zależności od płci odbiorcy. W języku polskim, podobnie jak w wielu innych językach, imiona własne nie zmieniają swojej formy w zależności od płci rozmówcy.

Zatem, tłumaczenie "Jesteś zabawna, Jane" jest poprawne i odpowiednie w tym kontekście, niezależnie od płci odbiorcy.


Agent

In [61]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [59]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\kajet\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kajet\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [65]:
def load_gender_examples_faiss(path: str):
    docs = []
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read().strip()
        blocks = raw.split("\n\n")  # oddzielenie przykładów

        for block in blocks:
            lines = block.strip().split("\n")
            if len(lines) < 4:
                continue  # pomiń niekompletne

            # Wydobycie danych
            speaker_line = next((l for l in lines if l.startswith("Mówca:")), None)
            listener_line = next((l for l in lines if l.startswith("Słuchacz:")), None)
            english_line = next((l for l in lines if l.startswith("Angielski:")), None)
            polish_line = next((l for l in lines if l.startswith("Polski:")), None)

            if not all([speaker_line, listener_line, english_line, polish_line]):
                continue  # pomiń błędne bloki

            speaker = speaker_line.replace("Mówca:", "").strip()
            listener = listener_line.replace("Słuchacz:", "").strip()
            en = english_line.replace("Angielski:", "").strip()
            pl = polish_line.replace("Polski:", "").strip()

            # Zbuduj treść dokumentu
            content = f"Mówca: {speaker}\nSłuchacz: {listener}\nAngielski: {en}\nPolski: {pl}"

            # Dodaj do dokumentów
            docs.append(Document(page_content=content, metadata={"en": en, "speaker": speaker, "listener": listener}))
    
    return docs

In [66]:
examples = load_gender_examples_faiss("examples/gender_examples.txt")
vectorstore = FAISS.from_documents(examples, embedding)

In [67]:
examples

[Document(metadata={'en': 'I am the best artist, are you a good artist?', 'speaker': 'mężczyzna', 'listener': 'kobieta'}, page_content='Mówca: mężczyzna\nSłuchacz: kobieta\nAngielski: I am the best artist, are you a good artist?\nPolski: Jestem najlepszym artystą, czy ty jesteś dobrą artystką?'),
 Document(metadata={'en': 'I am the best artist, are you a good artist?', 'speaker': 'kobieta', 'listener': 'mężczyzna'}, page_content='Mówca: kobieta\nSłuchacz: mężczyzna\nAngielski: I am the best artist, are you a good artist?\nPolski: Jestem najlepszą artystką, czy ty jesteś dobrym artystą?'),
 Document(metadata={'en': 'I am the best artist, are you a good artist?', 'speaker': 'mężczyzna', 'listener': 'mężczyzna'}, page_content='Mówca: mężczyzna\nSłuchacz: mężczyzna\nAngielski: I am the best artist, are you a good artist?\nPolski: Jestem najlepszym artystą, czy ty jesteś dobrym artystą?'),
 Document(metadata={'en': 'I am the best artist, are you a good artist?', 'speaker': 'kobieta', 'liste

In [70]:
def search_similar_gender_examples(speaker_gender: str, listener_gender: str, k=3) -> str:
    # Tworzymy zapytanie zawierające płeć mówiącego i słuchającego
    query_with_genders = f"Mówca: {speaker_gender} \nSłuchacz: {listener_gender}"
    
    # Wyszukiwanie podobnych przykładów w FAISS
    results = vectorstore.similarity_search(query_with_genders, k=k)
    
    # Zwracamy podobne dokumenty (tłumaczenia) w formacie tekstowym
    return "\n\n".join([doc.page_content for doc in results])

In [73]:
print(search_similar_gender_examples(speaker_gender="mężczyzna", listener_gender="kobieta", k=3))

Mówca: kobieta
Słuchacz: mężczyzna
Angielski: You cheated on me, now I am sad
Polski: Zdradziłeś mnie, teraz jestem smutna

Mówca: kobieta
Słuchacz: kobieta
Angielski: You cheated on me, now I am sad
Polski: Zdradziłaś mnie, teraz jestem smutna

Mówca: mężczyzna
Słuchacz: mężczyzna
Angielski: You cheated on me, now I am sad
Polski: Zdradziłeś mnie, teraz jestem smutny


In [74]:
import random

In [93]:
def load_examples(path: str):
    # Load all examples from the file
    examples = []
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read().strip()
        blocks = raw.split("\n\n")  # separate examples by empty lines

        for block in blocks:
            lines = block.strip().split("\n")
            if len(lines) < 4:
                continue  # skip incomplete examples

            speaker_line = next((l for l in lines if l.startswith("Mówca:")), None)
            listener_line = next((l for l in lines if l.startswith("Słuchacz:")), None)
            english_line = next((l for l in lines if l.startswith("Angielski:")), None)
            polish_line = next((l for l in lines if l.startswith("Polski:")), None)

            if not all([speaker_line, listener_line, english_line, polish_line]):
                continue  # skip if any of the required lines are missing

            speaker = speaker_line.replace("Mówca:", "").strip()
            listener = listener_line.replace("Słuchacz:", "").strip()
            en = english_line.replace("Angielski:", "").strip()
            pl = polish_line.replace("Polski:", "").strip()

            # Store the example with gender information
            examples.append({
                'Mówca': speaker,
                'Słuchacz': listener,
                'Angielski': en,
                'Polski': pl,
                
            })

    return examples

In [96]:
def get_random_examples_with_genders(examples, speaker_gender: str, listener_gender: str, num_examples: int):
    # Filter the examples based on the specified genders
    filtered_examples = [ex for ex in examples if ex['Mówca'] == speaker_gender and ex['Słuchacz'] == listener_gender]

    # If there are fewer examples than requested, return as many as we can
    if len(filtered_examples) < num_examples:
        print(f"Not enough examples, returning {len(filtered_examples)} examples.")
        num_examples = len(filtered_examples)

    # Randomly select the specified number of examples
    selected_examples = random.sample(filtered_examples, num_examples)

    return selected_examples

In [98]:
examples = load_examples('examples/gender_examples.txt')

# Get 3 random examples where the speaker is male and the listener is female
random_examples = get_random_examples_with_genders(examples, speaker_gender="mężczyzna", listener_gender="kobieta", num_examples=3)
random_examples

[{'Mówca': 'mężczyzna',
  'Słuchacz': 'kobieta',
  'Angielski': 'I almost killed him, and what did you do?',
  'Polski': 'Prawie go zabiłem, a ty co zrobiłaś?'},
 {'Mówca': 'mężczyzna',
  'Słuchacz': 'kobieta',
  'Angielski': 'You cheated on me, now I am sad',
  'Polski': 'Zdradziłaś mnie, teraz jestem smutny'},
 {'Mówca': 'mężczyzna',
  'Słuchacz': 'kobieta',
  'Angielski': "I wanted to paint my walls but you didn't borrow me paint",
  'Polski': 'Chciałem pomalować moje ściany ale nie pożyczyłaś mi farby'}]

In [99]:
explanation_template_with_examples = PromptTemplate(
    input_variables=["sentence", "speaker_gender", "listener_gender", "speaker_character", "examples"],
    template="""
Jesteś profesjonalnym tłumaczem z angielskiego na polski.

Zdanie do przetłumaczenia: "{sentence}"

Płeć mówcy: {speaker_gender}
Charakter mówcy: {speaker_character}

Płeć słuchacza: {listener_gender}

Przykłady podobnych tłumaczeń:
{examples}

Przeanalizuj zdanie pod kątem gramatycznym i semantycznym. 
Uwzględnij kontekst wypowiedzi, czyli jakiej płci jest mówca, jakiej płci jest słuchacz oraz czy mówca wypowiada się w charakterystyczny sposób.
Potem wytłumacz jak każda część zdania przekłada się na polską wersję uwzględniając kontekst.
Na końcu podaj tłumaczenie zdania na polski.

"""
)

In [100]:
explanation_translation_with_examples_chain = LLMChain(llm=clarin_llm, prompt=explanation_template_with_examples)

In [101]:
examples = load_examples('examples/gender_examples.txt')

# Get 3 random examples where the speaker is male and the listener is female
random_examples = get_random_examples_with_genders(examples, speaker_gender="kobieta", listener_gender="mężczyzna", num_examples=3)
random_examples

[{'Mówca': 'kobieta',
  'Słuchacz': 'mężczyzna',
  'Angielski': 'I am the best artist, are you a good artist?',
  'Polski': 'Jestem najlepszą artystką, czy ty jesteś dobrym artystą?'},
 {'Mówca': 'kobieta',
  'Słuchacz': 'mężczyzna',
  'Angielski': "I wanted to paint my walls but you didn't borrow me paint",
  'Polski': 'Chciałam pomalować moje ściany ale nie pożyczyłeś mi farby'},
 {'Mówca': 'kobieta',
  'Słuchacz': 'mężczyzna',
  'Angielski': 'I almost killed him, and what did you do?',
  'Polski': 'Prawie go zabiłam, a ty co zrobiłeś?'}]

In [102]:
print(explanation_translation_with_examples_chain.run(sentence=english_input, speaker_gender="kobieta", listener_gender="mężczyzna", examples=random_examples,
                                             speaker_character=None))

### Analiza Zdania

**Zdanie do przetłumaczenia:** "You're funny Jane but it'll cost you. Scratch my back, I'll scratch yours."

**Kontekst:**
- Płeć mówcy: kobieta
- Charakter mówcy: brak informacji o specyficznym charakterze
- Płeć słuchacza: mężczyzna

### Tłumaczenie i Wyjaśnienie

1. **You're funny Jane**:
   - Kontekst: Mówiąca (kobieta) zwraca się bezpośrednio do osoby o imieniu Jane, która jest również kobietą.
   - Polska wersja: "Jesteś zabawna, Jane."

2. **"but it'll cost you"**:
   - Kontekst: Mówiąca sugeruje, że bycie zabawnym ma swoje konsekwencje lub koszty dla Jane.
   - Polska wersja: "ale będzie cię to kosztować."

3. **"Scratch my back, I'll scratch yours"**:
   - Kontekst: To popularne powiedzenie oznaczające wzajemność lub współpracę. W tym kontekście mówiąca proponuje Jane, aby ta wykonała dla niej pewną przysługę, a ona odwdzięczy się tym samym.
   - Polska wersja: "Podrap mnie po plecach, ja podrapię ciebie."

### Pełne Tłumaczenie

Połączone tłumaczenia poszc

In [103]:
from langchain.agents import Tool

def gender_faiss_tool_func(sentence, speaker_gender, listener_gender) -> str:
    random_examples = get_random_examples_with_genders(examples, speaker_gender=speaker_gender, listener_gender=listener_gender, num_examples=3)
    prompt = f"""
    Zdanie do tłumacznenia: {sentence}

    Płeć mówiącego: {speaker_gender}
    Płeć słuchacza: {listener_gender}

    Przetłumacz podane zdanie na język polski uwzględniająć płeć mówcy oraz słuchacza.

    Przykłady podobnych tłumaczeń:
    {random_examples}
    """
    return clarin_llm(prompt)

gender_tool = Tool(
    name="GenderSearchTool",
    func=gender_faiss_tool_func,
    description="Searches gender-based translation examples"
)

In [104]:
from langchain.agents import initialize_agent, AgentType

In [114]:
agent = initialize_agent(
    tools=[gender_tool],  # dodaj inne jeśli chcesz
    llm=clarin_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
    
)

cos = agent({"input": """
             Jesteś profesjonalnym tłumaczem z angielskiego na polski.

Zdanie do przetłumaczenia: "{sentence}"

Płeć mówcy: {speaker_gender}
Charakter mówcy: {speaker_character}

Płeć słuchacza: {listener_gender}


Przeanalizuj zdanie pod kątem gramatycznym i semantycznym. 
Uwzględnij kontekst wypowiedzi, czyli jakiej płci jest mówca, jakiej płci jest słuchacz oraz czy mówca wypowiada się w charakterystyczny sposób.
Potem wytłumacz jak każda część zdania przekłada się na polską wersję uwzględniając kontekst.
Na końcu podaj tłumaczenie zdania na polski.


             """.format(sentence=english_input, speaker_gender="kobieta", listener_gender="mężczyzna", speaker_character=None)})



> Entering new AgentExecutor chain...
Najpierw przeanalizuję zdanie pod kątem gramatycznym i semantycznym, a następnie wezmę pod uwagę kontekst wypowiedzi. Później wyjaśnię, jak każda część zdania przekłada się na polską wersję, uwzględniając kontekst. Na koniec podam tłumaczenie zdania na polski.

**Analiza gramatyczna i semantyczna:**

1. Zdanie składa się z dwóch części połączonych spójnikiem "but" ("ale").
2. Pierwsza część zawiera komplement ("You're funny Jane") i ostrzeżenie ("it'll cost you").
3. Druga część to idiomatyczne powiedzenie ("Scratch my back, I'll scratch yours"), które może być interpretowane jako propozycja wymiany przysług.

**Kontekst wypowiedzi:**

- Mówca: kobieta
- Słuchacz: mężczyzna
- Charakter mówcy: brak informacji

**Tłumaczenie uwzględniające kontekst:**

1. "You're funny Jane":
   - Tłumaczenie: "Jesteś zabawna, Jane"
   - Kontekst: Użycie imienia sugeruje bliskość lub znajomość między mówcą a adresatem. W języku polskim również używa się imion w tak